# Read and clean a single file

In [1]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport
# %load_ext autoreload

project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'espbigdata'
directory = 'BigData'

## Read Metadata
-----
General info for each relevant column

In [2]:
file = 'gs://{}/{}/Data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


The final data must contains this and only this columns with that specific headers in order to standarize the data, this is a type of data dictionary but it also must include the data types (**schema**) of each field (**column**)

## Get raw data

In [2]:
raw_data = get_input_data(
    bucket = bucket, 
    initial_directory = directory, 
    filename = 'datos-abiertos-noviembre_2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15926 entries, 0 to 15925
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   15926 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  15926 non-null  object
 2   CODIGO DE LOCALIDAD                15926 non-null  int64 
 3   LOCALIDAD                          15926 non-null  object
 4   EDAD                               15926 non-null  int64 
 5   UNIDAD                             15926 non-null  object
 6   GENERO                             15926 non-null  object
 7   RED                                15926 non-null  object
 8   TIPO_INCIDENTE                     15926 non-null  object
 9   PRIORIDAD                          15926 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.2+ MB
None


It seems that this file contains already the fields, but need to rename them

### Rename Columns

In [3]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
     
    },
    inplace=True  #Npo sobre escribe 
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [4]:
raw_data['LOCALIDAD'].value_counts(dropna=False) ## dropna en true no muestra  los campos nullos en false los muestra por defecto viene en true


 Kennedy              2192
Engativa              1571
Suba                  1424
 Bosa                 1189
 Ciudad Bolivar       1123
 Puente Aranda         854
Rafael Uribe Uribe     835
Usaquen                808
Fontibon               787
San Cristobal          758
 Santa Fe              613
 Chapinero             577
 Usme                  577
Los Martires           560
 Tunjuelito            551
 Teusaquillo           546
 Barrios Unidos        520
 Antonio Nariño        338
 La Candelaria         102
 Sumapaz                 1
Name: LOCALIDAD, dtype: int64

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [5]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2192
Engativa              1571
Suba                  1424
Bosa                  1189
Ciudad Bolivar        1123
Puente Aranda          854
Rafael Uribe Uribe     835
Usaquen                808
Fontibon               787
San Cristobal          758
Santa Fe               613
Chapinero              577
Usme                   577
Los Martires           560
Tunjuelito             551
Teusaquillo            546
Barrios Unidos         520
Antonio Nariño         338
La Candelaria          102
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [6]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [12]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [8]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     7011
MASCULINO    4856
FEMENINO     4059
Name: GENERO, dtype: int64

In [9]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    5577
Inconsciente/Paro Cardiorrespiratorio        1587
Dificultad Respiratoria                      1400
Enfermo                                      1066
Convulsiones                                  979
Heridos                                       729
Lesiones personales                           675
Trastorno mental                              601
Intento de suicidio                           600
Dolor torácico                                577
Patología Gineco - obstétrica                 373
Síntomas gastrointestinales                   355
Caída                                         346
Accidente cerebro vascular                    318
Intoxicaciones                                137
SIN_DATO                                      122
Ideas de suicidio                              90
Solicitud Apoyo / Desacato                     83
Incendio estructural                           48
Muerte Natural                                 41


In [13]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [23]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [16]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [9]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_julio_2019.html')

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [17]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 15926
Number of rows after cleaning: 15886


In [21]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,1129118194,2019-11-01 00:01:16,18,Rafael Uribe Uribe,0,SIN_DATO,SIN_DATO,Sur,Lesiones personales,ALTA
1,1129198194,2019-11-01 00:14:45,11,Suba,47,Años,MASCULINO,Norte,Accidente cerebro vascular,ALTA
2,1129241194,2019-11-01 00:16:15,2,Chapinero,33,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,1129253194,2019-11-01 00:11:37,7,Bosa,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
4,1129258194,2019-11-01 00:16:28,11,Suba,43,Años,FEMENINO,Norte,Dolor torácico,ALTA


In [20]:
import  datetime
raw_data['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'] = raw_data['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].apply(lambda _: datetime.datetime.strptime(_,"%Y-%m-%d %H:%M:%S"))
#import  datetime
#raw_data.head()
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15886 entries, 0 to 15925
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   15886 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  15886 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   15886 non-null  int64         
 3   LOCALIDAD                          15886 non-null  object        
 4   EDAD                               15886 non-null  int64         
 5   UNIDAD                             15886 non-null  object        
 6   GENERO                             15886 non-null  object        
 7   RED                                15886 non-null  object        
 8   TIPO_INCIDENTE                     15886 non-null  object        
 9   PRIORIDAD                          15886 non-null  object        
dtypes: datetime64[ns](1), int64(2), ob

### Save the final table
use to_csv to save the table

In [22]:
# saving in a local directory
raw_data.to_csv('/home/jupyter/BigData/data/processed/' +'clean_llamadas_123_noviembre_2019.csv', encoding='latin1', sep=';', index=False)
#raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_julio_2019.csv', encoding='latin1', sep=';', index=False)